In [1]:
%%HTML
<h2>Implementing CART</h2>

### Getting Started

Before you get started, let's import a few packages that you will need. In addition, you will load two binary classification dataset - the spiral dataset and the <a href="https://archive.ics.uci.edu/ml/datasets/Ionosphere">ION</a> dataset. 

In [ ]:
<h2>Implementing CART</h2>

### Getting Started

Before you get started, let's import a few packages that you will need. In addition, you will load two binary classification dataset - the spiral dataset and the <a href="https://archive.ics.uci.edu/ml/datasets/Ionosphere">ION</a> dataset. 

In [ ]:
def spiraldata(N=300):
    r = np.linspace(1,2*np.pi,N) # generate a vector of "radius" values
    xTr1 = np.array([np.sin(2.*r)*r, np.cos(2*r)*r]).T # generate a curve that draws circles with increasing radius
    xTr2 = np.array([np.sin(2.*r+np.pi)*r, np.cos(2*r+np.pi)*r]).T
    xTr = np.concatenate([xTr1, xTr2], axis=0)
    yTr = np.concatenate([np.ones(N), -1 * np.ones(N)])
    xTr = xTr + np.random.randn(xTr.shape[0], xTr.shape[1])*0.2
    
    # Now sample alternating values to generate the test and train sets.
    xTe = xTr[::2,:]
    yTe = yTr[::2]
    xTr = xTr[1::2,:]
    yTr = yTr[1::2]
    
    return xTr, yTr, xTe, yTe

xTrSpiral, yTrSpiral, xTeSpiral, yTeSpiral = spiraldata(150)

In [ ]:
We can plot xTrSpiral to see the curve generated by the function above:

In [ ]:
plt.scatter(xTrSpiral[:,0], xTrSpiral[:,1],30,yTrSpiral)

In [ ]:
The following code loads the ION dataset.

In [ ]:
# load in some binary test data (labels are -1, +1)
data = loadmat("ion.mat")

# Load the training data
xTrIon  = data['xTr'].T
yTrIon  = data['yTr'].flatten()

# Load the test data
xTeIon  = data['xTe'].T
yTeIon  = data['yTe'].flatten()

In [2]:
%%HTML
<h2> Implement Regression Trees </h2>


<h3> Part One: Implement <code>sqimpurity</code> [Graded]</h3>

<p>First, implement the function <code>sqimpurity</code> which takes as input a vector of $n$ labels and outputs the corresponding squared loss impurity
    $$\sum_{i} (y_i-\bar{y}_i)^2 \textrm{ where: } \bar{y}_i=\frac{1}{n}\sum_{i} y_i$$
</p>

In [ ]:
def sqimpurity(yTr):
    """Computes the weighted variance of the labels
    
    Input:
        yTr:     n-dimensional vector of labels
    
    Output:
        impurity: weighted variance / squared loss impurity of this data set
    """
    
    N, = yTr.shape
    #print('N: ', N)
    assert N > 0 # must have at least one sample
    impurity = 0
    
    # YOUR CODE HERE
    
    #implement y_bar wich is the average label
    y_bar = np.sum(yTr)/N
    impurity = np.sum((yTr - y_bar)**2)

    
    
    return impurity

yTr = np.random.randn(100) # generate random labels
impurity = sqimpurity(yTr) # compute impurity
impurity

In [ ]:
def sqimpurity_test1():
    yTr = np.random.randn(100) # generate random labels
    impurity = sqimpurity(yTr) # compute impurity
    return np.isscalar(impurity)  # impurity should be scalar

def sqimpurity_test2():
    yTr = np.random.randn(100) # generate random labels
    impurity = sqimpurity(yTr) # compute impurity
    return impurity >= 0 # impurity should be nonnegative

def sqimpurity_test3():
    yTr = np.ones(100) # generate an all one vector as labels
    impurity = sqimpurity(yTr) # compute impurity
    return np.isclose(impurity, 0) # impurity should be zero since the labels are homogeneous

def sqimpurity_test4():
    yTr = np.arange(-5, 6) # generate a vector with mean zero
    impurity = sqimpurity(yTr) # compute impurity
    sum_of_squares = np.sum(yTr ** 2) 
    return np.isclose(impurity, sum_of_squares) # with mean zero, then the impurity should be the sum of squares

def sqimpurity_test5():
    yTr = np.random.randn(100) # generate random labels
    impurity = sqimpurity(yTr)
    impurity_grader = sqimpurity_grader(yTr)
    return np.isclose(impurity, impurity_grader)

runtest(sqimpurity_test1, 'sqimpurity_test1')
runtest(sqimpurity_test2, 'sqimpurity_test2')
runtest(sqimpurity_test3, 'sqimpurity_test3')
runtest(sqimpurity_test4, 'sqimpurity_test4')
runtest(sqimpurity_test5, 'sqimpurity_test5')

In [3]:
%%HTML
<h3> Part Two: Implement <code>sqsplit</code> [Graded]</h3>

<p>Now implement <code>sqsplit</code>, which takes as input a data set with labels and computes the best feature and cut-value of an optimal split based on the squared error impurity. The <code>sqsplit</code> function takes as input a data set of row vectors and a label vector and outputs a feature dimension, a cut threshold, and the impurity loss of this best split. The cut value should be the average of the values in the dimension where two datapoints are split. To find the best split, evaluate all possible splits and then search for the split that yields the minimum loss.</p> 

<p>Remember that we evaluate the quality of a split of a parent set $S_P$ into two sets $S_L$ and $S_R$ by the weighted impurity of the two branches, i.e.</p>

$\frac{\left|S_L\right|}{\left|S_P\right|}I\left(S_L\right)+\frac{\left|S_R\right|}{\left|S_P\right|}I\left(S_R\right)$

<p>In the case of the squared loss, this becomes:</p>

$\frac{1}{|S_P|}\sum_{(x,y)\in S_L}(y-\bar{y}_{S_L})^2 +\frac{1}{|S_P|}\sum_{(x,y)\in S_R}(y-\bar{y}_{S_R})^2$

<em>Note: Avoid splitting on datapoints with same value in a dimension.</em> 

In [ ]:
def sqsplit(xTr, yTr):
    """Finds the best feature, cut value, and loss value.
    
    Input:
        xTr:     n x d matrix of data points
        yTr:     n-dimensional vector of labels
    
    Output:
        feature:  index of the best cut's feature
        cut:      cut-value of the best cut
        bestloss: loss of the best cut
    """
    N,D = xTr.shape
    assert D > 0 # must have at least one dimension
    assert N > 1 # must have at least two samples
    
    bestloss = np.inf
    feature = np.inf
    cut = np.inf
    
    # YOUR CODE HERE
    #xTr = np.argsort(xTr, 0)
    for i in range(D):
        xTrindex = xTr[:,i].argsort()
        yTrsort = yTr[xTrindex] #yTr.argsort()
        xTrsort = xTr[xTrindex, i]
        #for y in xTrsort:
        for y in range(N-1):
        #for y in range(xTr.shape[0]):
            if xTrsort[y] != xTrsort[y+1]:
                #xTrL = xTr[:np.where(xTr[xTrsort,i]==y)[0][0]]
                #print('xTrL: ', xTrL)
                #xTrR = xTr[np.where(xTr[xTrsort]==y)[0][0]:]
                #print('xTrR: ', xTrR)
                yTrL = yTrsort[:y+1]   #yTr[:np.where(xTr[xTrsort,i] ==y)[0][0]]
               # print('yTrL: ', yTrL)
                yTrR = yTrsort[y+1:] #yTr[np.where(xTr[xTrsort,i] ==y)[0][0]:]
               # print('yTrR: ', yTrR)
   
                loss = sqimpurity(yTrL)+sqimpurity(yTrR)
                if loss < bestloss:
                    bestloss = loss 
                    cut = (xTrsort[y]+xTrsort[y+1])/2
                    feature = i
        #N, = yTrL.shape
    #print('shape', N)
   # print('feature: ', feature)
   # print('cut:', cut)
   # print('bestloss: ', bestloss)
    #return None 
    
    
    
    return feature, cut, bestloss

# x = np.array(range(1000)).reshape(-1,1)
# y = np.hstack([np.ones(500),-1*np.ones(500)]).T
# _, cut, _ = sqsplit(x, y)
# _, cut, _ 

In [ ]:
# The tests below check that your sqsplit function returns the correct values for several different input datasets

t0 = time.time()
fid, cut, loss = sqsplit(xTrIon,yTrIon)
t1 = time.time()

print('Elapsed time: {:0.2f} seconds'.format(t1-t0))
print("The best split is on feature 2 on value 0.304")
print("Your tree split on feature %i on value: %2.3f \n" % (fid,cut))

def sqsplit_test1():
    a = np.isclose(sqsplit(xor4, yor4)[2] / len(yor4), .25)
    b = np.isclose(sqsplit(xor3, yor3)[2] / len(yor3), .25)
    c = np.isclose(sqsplit(xor2, yor2)[2] / len(yor2), .25)
    return a and b and c

def sqsplit_test2():
    x = np.array(range(1000)).reshape(-1,1)
    y = np.hstack([np.ones(500),-1*np.ones(500)]).T
    _, cut, _ = sqsplit(x, y)
    return cut <= 500 or cut >= 499

def sqsplit_test3():
    fid, cut, loss = sqsplit(xor5,yor5)
    # cut should be 0.5 but 0 is also accepted
    return fid == 0 and (cut >= 0 or cut <= 1) and np.isclose(loss / len(yor5), 2/3)

runtest(sqsplit_test1,'sqsplit_test1')
runtest(sqsplit_test2,'sqsplit_test2')
runtest(sqsplit_test3,'sqsplit_test3')

In [4]:
%%HTML
<h3> Part Three: Implement <code>cart</code> [Graded]</h3>

In this section, you will implement the function <code>cart</code>, which returns a regression tree based on the minimum squared loss splitting rule. You should use the function <code>sqsplit</code> to make your splits. <p>Use the provided <code>TreeNode</code> class below to represent your tree. Note that the nature of CART trees implies that every node has exactly 0 or 2 children.</p>

In [5]:
%%HTML
<h4>Tree Structure</h4>

<p>We've provided a tree structure for you with distinct leaves and nodes. Leaves have two fields, parent (another node) and prediction (a numerical value).</p>

<strong>Nodes have six fields:</strong>

<ol>
<li> <b>left</b>: node describing left subtree </li>
<li> <b>right</b>: node describing right subtree </li>
<li> <b>feature</b>: index of feature to cut </li>
<li> <b>cut</b>: cutoff value c (<=c : left, and >c : right)</li>
<li> <b>prediction</b>: prediction at this node </li>. This should be the average of the labels at this node.
</ol>

In [ ]:
class TreeNode(object):
    """Tree class.
    (You don't need to add any methods or fields here but feel
    free to if you like. The tests will only reference the fields
    defined in the constructor below, so be sure to set these
    correctly.)
    """
    
    def __init__(self, left, right, feature, cut, prediction):
        self.left = left 
        self.right = right 
        self.feature = feature 
        self.cut = cut
        self.prediction = prediction 

In [ ]:
The following cell contains some examples of trees.

In [ ]:
# The following is a tree that predicts everything as zero ==> prediction 0
# In this case, it has no left or right children (it is a leaf node) ==> left = None, right = None
# The tree also do not split at any feature at any value ==> feature = None, cut = None, 
root = TreeNode(None, None, None, None, 0)


# The following that a tree with depth 2
# or a tree with one split 
# The tree will return a prediction of 1 if an example falls under the left subtree
# Otherwise it will return a prediction of 2
# To start, first create two leaf node
left_leaf = TreeNode(None, None, None, None, 1)
right_leaf = TreeNode(None, None, None, None, 2)

# Now create the parent or the root
# Suppose we split at feature 0 and cut of 1
# and the average prediction is 1.5
root2 = TreeNode(left_leaf, right_leaf, 0, 1 , 1.5)

# Now root2 is the tree we desired

In [ ]:
def cart(xTr,yTr):
    """Builds a CART tree.
    
    The maximum tree depth is defined by "maxdepth" (maxdepth=2 means one split).
    Each example can be weighted with "weights".

    Args:
        xTr:      n x d matrix of data
        yTr:      n-dimensional vector

    Returns:
        tree: root of decision tree
    """
    n,d = xTr.shape
    
    # YOUR CODE HERE
    
    #Can you split it?
    #if yes leftnode = cart(leftxTr, leftyTr), rightnode = cart(rightx, right y)
    
    '''
    new
    '''
    if (n < 2 ):
        return TreeNode(None,None,None, None, np.mean(yTr))
    
    feature, cut, bestloss = sqsplit(xTr,yTr)
    
    if feature != np.inf:
        #print('feature: ', feature)
        #print('cut: ', cut)
        #print('bestloss: ', bestloss)
        #print('xxx: ', xTr[:,feature]<=cut)
        #print('XXXTr: ', xTr[xTr[:,feature]<=cut])
        #print('yyyy: ', yTr[xTr[:,feature]<=cut])
        #xTrL = 
        #leftxTr = xTr[xTr[:,feature]<=cut]  -change
        #rightxTr = xTr[xTr[:,feature]>=cut] -change
        #print("xTr[:,feature]<=cut",xTr[:,feature]<=cut)
        #print("xTr[:,feature]<=cut----",xTr[xTr[:,feature]<=cut])
        leftxTr =  xTr[xTr[:,feature]<=cut]
        rightxTr = xTr[xTr[:,feature]>cut] ## removed =
        leftyTr = yTr[xTr[:,feature]<=cut]
        #rightyTr = yTr[xTr[:,feature]>=cut] -change
        rightyTr = yTr[xTr[:,feature]>cut]
        leftTree = cart(leftxTr,leftyTr)
        rightTree = cart(rightxTr,rightyTr)
        prediction = np.mean(yTr)
        #parentNode = TreeNode(leftTree, rightTree , feature, cut, prediction)-chnage
        return TreeNode(leftTree, rightTree , feature, cut, prediction)
    else:
        prediction = np.mean(yTr)
        #treeRoot = TreeNode(None,None,None, None, prediction)   # (leftTree, rightTree , feature, cut, predict)-change
        return TreeNode(None,None,None, None, prediction)   # (leftTree, rightTree , feature, cut, predict)
    
    #print('Tree: ', treeRoot)
    #print('predict', predict)
    
    #return treeNode
    
    #raise NotImplementedError()


In [ ]:
# The tests below check that your implementation of cart  returns the correct predicted values for a sample dataset

#test case 1
def cart_test1():
    t=cart(xor4,yor4)
    return DFSxor(t)

#test case 2
def cart_test2():
    y = np.random.rand(16);
    t = cart(xor4,y);
    yTe = DFSpreds(t)[:];
    # Check that every label appears exactly once in the tree
    y.sort()
    yTe.sort()
    return np.all(np.isclose(y, yTe))

#test case 3
def cart_test3():
    xRep = np.concatenate([xor2, xor2])
    yRep = np.concatenate([yor2, 1-yor2])
    t = cart(xRep, yRep)
    return DFSxorUnsplittable(t)

#test case 4
def cart_test4():
    X = np.ones((5, 2)) # Create a dataset with identical examples
    y = np.ones(5)
    
    # On this dataset, your cart algorithm should return a single leaf
    # node with prediction equal to 1
    t = cart(X, y)
    
    # t has no children
    children_check = (t.left is None) and (t.right is None) 
    
    # Make sure t does not cut any feature and at any value
    feature_check = (t.feature is None) and (t.cut is None)
    
    # Check t's prediction
    prediction_check = np.isclose(t.prediction, 1)
    return children_check and feature_check and prediction_check

#test case 5
def cart_test5():
    X = np.arange(4).reshape(-1, 1)
    y = np.array([0, 0, 1, 1])

    t = cart(X, y) # your cart algorithm should generate one split
    
    # check whether you set t.feature and t.cut to something
    return t.feature is not None and t.cut is not None



runtest(cart_test1,'cart_test1')
runtest(cart_test2,'cart_test2')
runtest(cart_test3,'cart_test3')
runtest(cart_test4,'cart_test4')
runtest(cart_test5,'cart_test5')

In [6]:
%%HTML
<h3> Part Four: Implement <code>evaltree</code> [Graded]</h3>

<p>Implement the function <code>evaltree</code>, which evaluates a decision tree on a given test data set.</p>

In [ ]:
def evaltree(root,xTe):
    """Evaluates xTe using decision tree root.
    
    Input:
        root: TreeNode decision tree
        xTe:  n x d matrix of data points
    
    Output:
        pred: n-dimensional vector of predictions
    """
    # YOUR CODE HERE
    
    pred = []
    '''
    for i in rsnge N
        XTE[I]
    root has a cut
    while loop
        xte  000110
        cut  - 0.5
        f= 2.
        root .lrft = root
    
    
    '''
    for i in range(N):
        while root.cut !=None:
            
            if root.feature == None and root.cut == None:
        pred.append(root.prediction)
        return root.prediction
    
    if xTe[root.feature] <= root.cut:
        evaltree(root.left, xTe)
    if xTe[root.feature] > root.cut:
        evaltree(root.right,xTe)
    return pred


#         if root.feature == None and root.cut == None:
#         pred.append(root.prediction)
#         return root.prediction
    
#     if xTe[root.feature] <= root.cut:
#         evaltree(root.left, xTe)
#     if xTe[root.feature] > root.cut:
#         evaltree(root.right,xTe)
#     return pred
#         evaltree(root.left
    
#     while root.left:
#         if xTr[:root.feature] <= root.cut:
#             root = root.left
#         else:
#             root = root.right
#     return root.prediction
    
#     if root.feature:
#         left = evaltree(root.left,xTe[:, feature]<=root.cut)
#         right = evaltree(root.right, xTe[:, feature]>root.cut)
#         print('left: ', left)
#         print('right: ', right)
#         return left, right

    #print('rootfeature: ', root.feature)
    #print('rootleft: ', root ==True)
    #print('rootcut: ', root.cut)
    #print('rootpred: ', root.right.prediction)
    print('xTe: ', xTe[0])
    
    pred = root.prediction
    print('pred: ', pred)
    
    #return pred
    raise NotImplementedError()
    
a = TreeNode(None, None, None, None, 1)
b = TreeNode(None, None, None, None, -1)
c = TreeNode(None, None, None, None, 0)
d = TreeNode(None, None, None, None, -1)
e = TreeNode(None, None, None, None, -1)
x = TreeNode(a, b, 0, 10, 0)
y = TreeNode(x, c, 0, 20, 0)
z = TreeNode(d, e, 0, 40, 0)
t = TreeNode(y, z, 0, 30, 0)
# Check that the custom tree evaluates correctly
print(evaltree(t, np.array([[45, 35, 25, 15, 5]]).T),np.array([-1, -1, 0, -1, 1]))


In [ ]:
# The following tests check that your implementation of evaltree returns the correct predictions for two sample trees

t0 = time.time()
root = cart(xTrIon, yTrIon)
t1 = time.time()

tr_err   = np.mean((evaltree(root,xTrIon) - yTrIon)**2)
te_err   = np.mean((evaltree(root,xTeIon) - yTeIon)**2)

print("Elapsed time: %.2f seconds" % (t1-t0))
print("Training RMSE : %.2f" % tr_err)
print("Testing  RMSE : %.2f \n" % te_err)

#test case 1
def evaltree_test1():
    t = cart(xor4,yor4)
    xor4te = xor4 + (np.sign(xor4 - .5) * .1)
    inds = np.arange(16)
    np.random.shuffle(inds)
    # Check that shuffling and expanding the data doesn't affect the predictions
    return np.all(np.isclose(evaltree(t, xor4te[inds,:]), yor4[inds]))

#test case 2
def evaltree_test2():
    a = TreeNode(None, None, None, None, 1)
    b = TreeNode(None, None, None, None, -1)
    c = TreeNode(None, None, None, None, 0)
    d = TreeNode(None, None, None, None, -1)
    e = TreeNode(None, None, None, None, -1)
    x = TreeNode(a, b, 0, 10, 0)
    y = TreeNode(x, c, 0, 20, 0)
    z = TreeNode(d, e, 0, 40, 0)
    t = TreeNode(y, z, 0, 30, 0)
    # Check that the custom tree evaluates correctly
    return np.all(np.isclose(
            evaltree(t, np.array([[45, 35, 25, 15, 5]]).T),
            np.array([-1, -1, 0, -1, 1])))

runtest(evaltree_test1,'evaltree_test1')
runtest(evaltree_test2,'evaltree_test2')

In [7]:
%%HTML
<h3> Visualize Your Tree</h3>

<p>The following code defines a function <code>visclassifier()</code>, which plots the decision boundary of a classifier in 2 dimensions. Execute the following code to see what the decision boundary of your tree looks like on the ion data set. </p>

In [ ]:
def visclassifier(fun,xTr,yTr,w=None,b=0):
    """
    visualize decision boundary
    Define the symbols and colors we'll use in the plots later
    """

    yTr = np.array(yTr).flatten()
    

    symbols = ["ko","kx"]
    marker_symbols = ['o', 'x']
    mycolors = [[0.5, 0.5, 1], [1, 0.5, 0.5]]
    # get the unique values from labels array
    classvals = np.unique(yTr)

    plt.figure()

    # return 300 evenly spaced numbers over this interval
    res=300
    xrange = np.linspace(min(xTr[:, 0]), max(xTr[:, 0]),res)
    yrange = np.linspace(min(xTr[:, 1]), max(xTr[:, 1]),res)
    
    # repeat this matrix 300 times for both axes
    pixelX = repmat(xrange, res, 1)
    pixelY = repmat(yrange, res, 1).T

    
    xTe = np.array([pixelX.flatten(), pixelY.flatten()]).T

    # test all of these points on the grid
    testpreds = fun(xTe)
    
    # reshape it back together to make our grid
    Z = testpreds.reshape(res, res)
    # Z[0,0] = 1 # optional: scale the colors correctly
    
    # fill in the contours for these predictions
    plt.contourf(pixelX, pixelY, np.sign(Z), colors=mycolors)

    # creates x's and o's for training set
    for idx, c in enumerate(classvals):
        plt.scatter(xTr[yTr == c,0],
            xTr[yTr == c,1],
            marker=marker_symbols[idx],
            color='k'
            )
    
    if w is not None:
        w = np.array(w).flatten()
        alpha = -1 * b / (w ** 2).sum()
        plt.quiver(w[0] * alpha, w[1] * alpha,
            w[0], w[1], linewidth=2, color=[0,1,0])

    plt.axis('tight')
    # shows figure and blocks
    plt.show()

tree=cart(xTrSpiral,yTrSpiral) # compute tree on training data 
visclassifier(lambda X:evaltree(tree,X), xTrSpiral, yTrSpiral)
print("Training error: %.4f" % np.mean(np.sign(evaltree(tree,xTrSpiral)) != yTrSpiral))
print("Testing error:  %.4f" % np.mean(np.sign(evaltree(tree,xTeSpiral)) != yTeSpiral))

In [ ]:
def onclick_cart(event):
    """
    Visualize cart, including new point
    """
    global xTraining,labels
    # create position vector for new point
    pos=np.array([[event.xdata,event.ydata]]) 
    if event.key == 'shift': # add positive point
        color='or'
        label=1
    else: # add negative point
        color='ob'
        label=-1    
    xTraining = np.concatenate((xTraining,pos), axis = 0)
    labels.append(label);
    marker_symbols = ['o', 'x']
    classvals = np.unique(labels)
        
    mycolors = [[0.5, 0.5, 1], [1, 0.5, 0.5]]
    
    # return 300 evenly spaced numbers over this interval
    res=300
    xrange = np.linspace(0, 1,res)
    yrange = np.linspace(0, 1,res)

    
    # repeat this matrix 300 times for both axes
    pixelX = repmat(xrange, res, 1)
    pixelY = repmat(yrange, res, 1).T

    xTe = np.array([pixelX.flatten(), pixelY.flatten()]).T

    # get decision tree
    tree=cart(xTraining,np.array(labels).flatten())
    fun = lambda X:evaltree(tree,X)
    # test all of these points on the grid
    testpreds = fun(xTe)
    
    # reshape it back together to make our grid
    Z = testpreds.reshape(res, res)
    # Z[0,0] = 1 # optional: scale the colors correctly
    
    plt.cla()    
    plt.xlim((0,1))
    plt.ylim((0,1))
    # fill in the contours for these predictions
    plt.contourf(pixelX, pixelY, np.sign(Z), colors=mycolors)
    
    for idx, c in enumerate(classvals):
        plt.scatter(xTraining[labels == c,0],
            xTraining[labels == c,1],
            marker=marker_symbols[idx],
            color='k'
            )
    plt.show()
    
        
xTraining= np.array([[5,6]])
labels = [1]
%matplotlib notebook
fig = plt.figure()
plt.xlim(0,1)
plt.ylim(0,1)
cid = fig.canvas.mpl_connect('button_press_event', onclick_cart)
plt.title('Click to add positive points and use shift-click to add negative points.')